In [35]:
import pandas as pd
import os

In [36]:
A=list([file.split(".")[0] for file in os.listdir('./') if ".txt" in file ])
A.remove('')
A.sort()

In [37]:
Base_Table=pd.read_table(A[0]+".txt",delimiter='\t',header=None)
Final_table={}
Final_table['Taxonomy']=list(Base_Table[0])
for Sample in A:
    Final_table[Sample]=[]

In [38]:
for Sample in A:
    Temp_Table=pd.read_table(Sample+".txt",delimiter='\t',header=None).set_index(0)
    for taxon in Final_table['Taxonomy']:
        Final_table[Sample].append(Temp_Table.loc[taxon,1])
        

In [39]:
FT=pd.DataFrame(Final_table)

In [40]:
FT.set_index("Taxonomy",inplace=True)

In [41]:
FT=FT[FT.sum(axis=1)>10000]

In [42]:
FT.to_csv('Clean_Table_Not_Normalized.csv')

In [43]:
FT.reset_index(inplace=True)

In [44]:
Rank=FT.groupby(list(map(len,FT['Taxonomy'].str.split('|'))))

In [45]:
Domain=Rank.get_group(1).set_index('Taxonomy')

In [46]:
Phylum=Rank.get_group(2).set_index("Taxonomy")

In [47]:
Class=Rank.get_group(3).set_index("Taxonomy")

In [48]:
Order=Rank.get_group(4).set_index("Taxonomy")

In [49]:
Family=Rank.get_group(5).set_index("Taxonomy")

In [50]:
Group=Rank.get_group(6).set_index("Taxonomy")

In [51]:
Species=Rank.get_group(7).set_index("Taxonomy")

In [52]:
Inds=["Control_328","Control_330","Control_327",
      "DSS_334","DSS_339","RCE_322","RCE_318","RCE_319"
      ,"RCE_plus_DSS_305","RCE_plus_DSS_313","RCE_plus_DSS_301"]

In [53]:
Phylum_Inds=Phylum.loc[:,Inds]
Class_Inds=Class.loc[:,Inds]
Order_Inds=Order.loc[:,Inds]
Family_Inds=Family.loc[:,Inds]
Group_Inds=Group.loc[:,Inds]
Species_Inds=Species.loc[:,Inds]

In [54]:
To_Write=[Phylum,Class,Order,Family,Group,Species,Phylum_Inds,Class_Inds,Order_Inds,Family_Inds,Group_Inds,Species_Inds]

In [55]:
Names="Phylum,Class,Order,Family,Group,Species,Phylum_Inds,Class_Inds,Order_Inds,Family_Inds,Group_Inds,Species_Inds".split(",")

In [56]:
for index,file in enumerate(To_Write):
    file.to_csv("Cleaned_Tables/"+Names[index]+'.csv')

In [57]:
Species_TSS=Species.copy()
Group_TSS=Group.copy()
Family_TSS=Family.copy()
Phylum_TSS=Phylum.copy()


In [58]:
for i in range(Species_TSS.shape[1]):
    Species_TSS.iloc[:,i]=Species_TSS.iloc[:,i]/Species_TSS.iloc[:,i].sum()*100
for i in range(Group_TSS.shape[1]):
    Group_TSS.iloc[:,i]=Group_TSS.iloc[:,i]/Group_TSS.iloc[:,i].sum()*100
for i in range(Family_TSS.shape[1]):
    Family_TSS.iloc[:,i]=Family_TSS.iloc[:,i]/Family_TSS.iloc[:,i].sum()*100
for i in range(Phylum_TSS.shape[1]):
    Phylum_TSS.iloc[:,i]=Phylum_TSS.iloc[:,i]/Phylum_TSS.iloc[:,i].sum()*100

In [59]:
Species_TSS.to_csv("Species_TSS.csv")
Group_TSS.to_csv("Group_TSS.csv")
Family_TSS.to_csv("Family_TSS.csv")
Phylum_TSS.to_csv("Phylum_TSS.csv")


In [60]:
Group_TSS.iloc[:,2].sum()

99.99999999999999

In [61]:
Group_TSS.iloc[:,i]/Group_TSS.iloc[:,i].sum()

Taxonomy
d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Muribaculaceae|g__Duncaniella                  0.171380
d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Muribaculaceae|g__Muribaculum                  0.236730
d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Muribaculaceae|g__Sodaliphilus                 0.007929
d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides                  0.097661
d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes                     0.031550
                                                                                                                 ...   
d__Bacteria|p__Fusobacteria|c__Fusobacteriia|o__Fusobacteriales|f__Fusobacteriaceae|g__Fusobacterium           0.000619
d__Bacteria|p__Fusobacteria|c__Fusobacteriia|o__Fusobacteriales|f__Leptotrichiaceae|g__Leptotrichia            0.000228
d__Eukaryota|k__Metazoa|p__Chor